# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
1. Change to exercise08 repository

2. Start docker <br>
docker-compose up -d

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to hdfs :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to hdfs needs to be done only once and it might take 1-2 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Preprocessing commands
In your newly created notebook run these commands in order to have the dataset into an RDD:

In [1]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
raw_data = sc.textFile(path)
dataset = raw_data.map(json.loads).cache()

After that you will be able to run the queries of the moodle question of this week. The RDD that you have to perform your queries on is the ```dataset``` one. For example, the following command returns one element of the dataset:

In [2]:
dataset.take(1)

[{'guess': 'Norwegian',
  'target': 'Norwegian',
  'country': 'AU',
  'choices': ['Maori', 'Mandarin', 'Norwegian', 'Tongan'],
  'sample': '48f9c924e0d98c959d8a6f1862b3ce9a',
  'date': '2013-08-19'}]

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [2]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

If you want to have a part of the dataset in order to run faster experiments then run the cell below and run your queries against the ```dataset_part``` RDD. However your final answer, the format of your final query and the productivity time should be run/measured against the whole dataset.

In [4]:
dataset_part = dataset.sample(False, 0.0001).cache()

## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [5]:
start_exercise()

In [6]:
dataset.filter(lambda x: x['target']==x['guess']=='Russian').count()

290818

In [7]:
finish_exercise()

This exercise took 86s


## Assignment 2
Return the number of distinct "target" languages.

In [8]:
start_exercise()

In [9]:
dataset.map(lambda x: x['target']).distinct().count()

78

In [10]:
finish_exercise()

This exercise took 58s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [11]:
start_exercise()

In [12]:
dataset.filter(lambda x: x["target"] == x["guess"]).sortBy(lambda x: x["target"] + x["country"] + x["date"], ascending=True).map(lambda x: x["sample"]).take(3)

['00b85faa8b878a14f8781be334deb137',
 'efcd813daec1c836d9f030b30caa07ce',
 '13722ceed1eede7ba597ade9b4cb9807']

In [13]:
finish_exercise()

This exercise took 193s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [10]:
start_exercise()

In [11]:
dataset.groupBy(lambda x:(x["country"], x["target"])).mapValues(lambda x: len(x)).sortBy(lambda x: x[1], ascending=False).map(lambda x: x[1]).take(3)

[112934, 112007, 110919]

In [12]:
finish_exercise() 

This exercise took 365s


## Assignment 5
Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [13]:
start_exercise()

In [29]:
round(dataset.filter(lambda x: x["guess"] == x["target"] == x["choices"][0]).count() / dataset.count(), 4)

0.256

In [15]:
finish_exercise()

This exercise took 245s


## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first three languages.

In [16]:
start_exercise()

In [25]:
dataset.groupBy(lambda x: x["target"]).mapValues(lambda x: sum(1 if y["guess"] == y["target"] else 0 for y in x) / len(x)).sortBy(lambda x: x[1], ascending=False).map(lambda x: x[0]).take(3)

['French', 'German', 'Spanish']

In [26]:
finish_exercise()

This exercise took 3572s


## Assignment 7
Return the number games played on the latest day.

In [22]:
start_exercise()

In [27]:
dataset.map(lambda x: (x["date"], 1)).reduceByKey(lambda a,b: a+b).sortByKey(ascending=False).map(lambda x: x[1]).take(1)

[65653]

In [24]:
finish_exercise()

This exercise took 676s
